In [2]:
import pandas as pd

In [3]:
# annotations_df = pd.read_csv('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat/full_annotations_df.csv')
annotations_df = pd.read_csv('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat_predictor_splits/full.csv')

for col in annotations_df.columns:
    if 'extraction' in col:
        annotations_df[col] = annotations_df[col].apply(lambda x: x.replace('[', '').replace(']', '').split(',')[0])

# {I, A, T} extraction to cluster
i_e2c = annotations_df[['dialogue', 'timestamp', 'case', 'cvid', 'instrument', 'instrument-extraction']].replace('NONE', None).dropna()
a_e2c = annotations_df[['dialogue', 'timestamp', 'case', 'cvid', 'action', 'action-extraction']].replace('NONE', None).dropna()
t_e2c = annotations_df[['dialogue', 'timestamp', 'case', 'cvid', 'tissue', 'tissue-extraction']].replace('NONE', None).dropna()

print(i_e2c.shape)
print(a_e2c.shape)
print(t_e2c.shape)
print(i_e2c.shape[0] + a_e2c.shape[0] + t_e2c.shape[0])

(344, 6)
(875, 6)
(512, 6)
1731


In [7]:
annotations_df['num_not_nan'] = annotations_df.apply(lambda row: 1*(row['instrument'] != 'NONE') + 1*(row['action'] != 'NONE') + 1*(row['tissue'] != 'NONE'), axis=1)

In [12]:
annotations_df['instrument-extraction'].nunique(), annotations_df['action-extraction'].nunique(), annotations_df['tissue-extraction'].nunique()

(114, 1107, 251)

In [13]:
annotations_df['instrument'].nunique(), annotations_df['action'].nunique(), annotations_df['tissue'].nunique()

(22, 32, 20)

In [8]:
annotations_df['num_not_nan'].value_counts()

num_not_nan
0    2389
1    1405
2     377
3      39
Name: count, dtype: int64

In [11]:
377+39

416

In [4]:
t_e2c[t_e2c['dialogue'].str.contains('spermatic cord')]

,dialogue,tissue,tissue-extraction
335,"youre not gonna follow the spermatic cord, you...",seminal_vesicle_vas,spermatic cord


## Figure

In [7]:
annotations_df[(annotations_df['instrument'] != 'NONE') & (annotations_df['action'] != 'NONE') & (annotations_df['tissue'] != 'NONE')].iloc[3]

dialogue                                           just buzz those vessels
timestamp                                                         15:52:48
case                                                                     8
cvid                                                     c8_s0_0-20-29.avi
procedure                                                      nephrectomy
procedure_defn           Nephrectomy is a surgical procedure that invol...
task                                                            dissecting
task_defn                Dissecting involves carefully separating tissu...
instrument                                                   energy_device
action                                                           coagulate
tissue                                                 general_vasculature
instrument-extraction                                       electrocautery
action-extraction                                                     buzz
tissue-extraction        

In [8]:
annotations_df[(annotations_df['action'] != 'NONE') & (annotations_df['tissue'] != 'NONE')].iloc[30]

dialogue                                            stay above that tendon
timestamp                                                         15:52:45
case                                                                     8
cvid                                                     c8_s0_0-20-26.avi
procedure                                                      nephrectomy
procedure_defn           Nephrectomy is a surgical procedure that invol...
task                                                            dissecting
task_defn                Dissecting involves carefully separating tissu...
instrument                                                            NONE
action                                                       control_depth
tissue                                                      muscle_capsule
instrument-extraction                                                 NONE
action-extraction                                               stay above
tissue-extraction        

In [9]:
annotations_df[(annotations_df['instrument'] != 'NONE') & (annotations_df['action'] != 'NONE') & (annotations_df['tissue'] != 'NONE')].iloc[4]

dialogue                                           get L hand below ureter
timestamp                                                         14:41:07
case                                                                    10
cvid                                                    c10_s0_0-30-28.avi
procedure                                                      nephrectomy
procedure_defn           Nephrectomy is a surgical procedure that invol...
task                                                            Dissection
task_defn                Dissection involves carefully separating and i...
instrument                                                       left_hand
action                                                       control_depth
tissue                                                              ureter
instrument-extraction                                            left hand
action-extraction                                                get below
tissue-extraction        

## Top 10 Terms

In [35]:
from collections import Counter

i_e2c_terms = i_e2c.groupby('instrument').agg(list).reset_index()
a_e2c_terms = a_e2c.groupby('action').agg(list).reset_index()
t_e2c_terms = t_e2c.groupby('tissue').agg(list).reset_index()

# i_e2c_terms['instrument-extraction'] = i_e2c_terms['instrument-extraction'].apply(lambda x: [item for sublist in x for item in sublist])
# a_e2c_terms['action-extraction'] = a_e2c_terms['action-extraction'].apply(lambda x: [item for sublist in x for item in sublist])
# t_e2c_terms['tissue-extraction'] = t_e2c_terms['tissue-extraction'].apply(lambda x: [item for sublist in x for item in sublist])

i_e2c_terms['extraction_term_counts'] = i_e2c_terms['instrument-extraction'].apply(lambda x: dict(sorted(Counter(x).items(), key=lambda item: item[1], reverse=True)) )
a_e2c_terms['extraction_term_counts'] = a_e2c_terms['action-extraction'].apply(lambda x: dict(sorted(Counter(x).items(), key=lambda item: item[1], reverse=True)) )
t_e2c_terms['extraction_term_counts'] = t_e2c_terms['tissue-extraction'].apply(lambda x: dict(sorted(Counter(x).items(), key=lambda item: item[1], reverse=True)) )

i_e2c_terms['extraction_term_counts'] = i_e2c_terms['extraction_term_counts'].apply(lambda x: [f"{k} ({v})" for k, v in x.items()])
a_e2c_terms['extraction_term_counts'] = a_e2c_terms['extraction_term_counts'].apply(lambda x: [f"{k} ({v})" for k, v in x.items()])
t_e2c_terms['extraction_term_counts'] = t_e2c_terms['extraction_term_counts'].apply(lambda x: [f"{k} ({v})" for k, v in x.items()])


In [36]:
i_e2c_terms[['instrument', 'extraction_term_counts']].to_csv('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat/instrument_cluster_counts.csv', index=False)
a_e2c_terms[['action', 'extraction_term_counts']].to_csv('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat/action_cluster_counts.csv', index=False)
t_e2c_terms[['tissue', 'extraction_term_counts']].to_csv('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat/tissue_cluster_counts.csv', index=False)

In [38]:
i_e2c_terms['extraction_top10_terms'] = i_e2c_terms['instrument-extraction'].apply(lambda x: [item[0] for item in Counter(x).most_common(10)])
a_e2c_terms['extraction_top10_terms'] = a_e2c_terms['action-extraction'].apply(lambda x: [item[0] for item in Counter(x).most_common(10)])
t_e2c_terms['extraction_top10_terms'] = t_e2c_terms['tissue-extraction'].apply(lambda x: [item[0] for item in Counter(x).most_common(10)])

top10_rows = []

In [22]:
# instruments
for i in range(len(i_e2c_terms)):
    row = i_e2c_terms.iloc[i]
    print(row['instrument'])
    print(str(row['extraction_top10_terms']).replace("'", '').replace('[', '').replace(']', ''))
    print('-'*100)
    top10_rows.append({
        'IAT': 'instrument',
        'IAT_term': row['instrument'],
        'IAT_top10_terms': row['extraction_top10_terms']
    })

energy_device
electrocautery, hook, energy device, bovie, coagulation tool, clip, coagulator, monopolar, bipolar, vessel sealer
----------------------------------------------------------------------------------------------------
fourth_arm
fourth arm, 4th arm, stitch, retractor, scissors
----------------------------------------------------------------------------------------------------
left_hand
left hand, left arm, right hand, scissors, 4th arm, fourth arm, left hand instrument
----------------------------------------------------------------------------------------------------
needle
needle, needles, balloon, suture
----------------------------------------------------------------------------------------------------
stitch
stitch, stitches, chromic stitch, foley
----------------------------------------------------------------------------------------------------
suture_material
suture, sutures, vicryl, 3-0 suture, vicyl stitch
-----------------------------------------------------------

In [12]:
# actions
for i in range(len(a_e2c_terms)):
    row = a_e2c_terms.iloc[i]
    print(row['action'])
    print(row['extraction_top10_terms'])
    print('-'*100)
    top10_rows.append({
        'IAT': 'action',
        'IAT_term': row['action'],
        'IAT_top10_terms': row['extraction_top10_terms']
    })

adjust
['get right', 'slow down', 'go slow', 'narrow down', 'match up', 'take', 'increase', 'go slower', 'go', 'ease']
----------------------------------------------------------------------------------------------------
adjust_direction
['go lateral', 'be more midline', 'Aim back up', 'err on the lateral sides', 'come from the lateral side', 'go obliquely', 'avoid', 'come obliquely', 'come from the sides', 'go cephalad more']
----------------------------------------------------------------------------------------------------
apply_traction
['grab', "don't pull", 'pull down', 'pull', 'pull back', 'hold', 'take', 'Press', 'stop', 'pull up']
----------------------------------------------------------------------------------------------------
coagulate
['buzz', 'sweep', 'Buzz', "don't buzz blindly", 'be patient', 'burn', 'coag', 'buzz on that crease', 'give a little buzz', 'bovie']
----------------------------------------------------------------------------------------------------
continue_

In [13]:
# tissues
for i in range(len(t_e2c_terms)):
    row = t_e2c_terms.iloc[i]
    print(row['tissue'])
    print(row['extraction_top10_terms'])
    print('-'*100)
    top10_rows.append({
        'IAT': 'tissue',
        'IAT_term': row['tissue'],
        'IAT_top10_terms': row['extraction_top10_terms']
    })

adrenal_kidney_region
['kidney', 'liver', 'psoas', 'adrenal', 'lateral edge of adrenal', 'cyst', 'adrenal vessels', 'lower pole', 'fascia of psoas', 'psoas sheets']
----------------------------------------------------------------------------------------------------
bladder
['bladder', 'bladder neck', 'posterior bladder neck', 'anterior bladder neck', 'prostate', 'bladder fibers', 'central suture', 'vessels', 'abdominal fat', 'needle catheter']
----------------------------------------------------------------------------------------------------
general_vasculature
['vessel', 'vessels', 'veins', 'aorta', 'arteries', 'vein', 'pedicles', 'blood vessel', 'lateral structure', 'arterial pumper']
----------------------------------------------------------------------------------------------------
gi_tract
['mucosa', 'bowel', 'rectum', 'diverticuli', 'mucosal edge', 'colon', 'rectal integrity', 'inner mucosa', 'bowel wall', 'ileostomy']
------------------------------------------------------------

In [14]:
top10_df = pd.DataFrame(top10_rows)
top10_df.set_index(['IAT', 'IAT_term'])

IAT_top10_terms
IAT        IAT_term                                                                     
instrument energy_device               [electrocautery, hook, energy device, bovie, c...
           fourth_arm                  [fourth arm, 4th arm, stitch, retractor, sciss...
           left_hand                   [left hand, left arm, right hand, scissors, 4t...
           needle                                     [needle, needles, balloon, suture]
           stitch                              [stitch, stitches, chromic stitch, foley]
           suture_material             [suture, sutures, vicryl, 3-0 suture, vicyl st...
action     adjust                      [get right, slow down, go slow, narrow down, m...
           adjust_direction            [go lateral, be more midline, Aim back up, err...
           apply_traction              [grab, don't pull, pull down, pull, pull back,...
           coagulate                   [buzz, sweep, Buzz, don't buzz blindly, be pat...
           continue_action             [give, do, try, use, exhaust, create more, do ...
           control_depth               [go down, go higher, come down, go deeper, kee...
           cut                         [cut, cut down, cold cut, do one thing, cut ju...
           dissect                     [get off, deliver, free up, take out, take awa...
           ensure_safety               [be careful, rip, be gentle, don't cut, rip ca...
           grasp                       [grab, take, regrab, take it, hold, hang on, t...
           inspect                     [see, watch, do, look, reduce, see below, see ...
           navigate_space              [come, travel more, come back, get out, back u...
           open                        [open, open up, open more, open up a little bi...
           perform_sweeping_maneuvers  [sweep, sweep down, sweep off, don't sweep, sw...
           release_instrument          [drop, leave, leave it, drop down, leave it be...
           rotate                                          [rotate, reverse, Rotate, do]
           spread                      [separate, widen, make it wide, extend distall...
           stop                                                 [stop, stop short, Stop]
           suture                      [throw, sew, don't pull, loop up, take stitch,...
           turn                        [turn, flip, flip up, U it back, Turn more, cu...
tissue     adrenal_kidney_region       [kidney, liver, psoas, adrenal, lateral edge o...
           bladder                     [bladder, bladder neck, posterior bladder neck...
           general_vasculature         [vessel, vessels, veins, aorta, arteries, vein...
           gi_tract                    [mucosa, bowel, rectum, diverticuli, mucosal e...
           major_veins                 [vein, veins, tissue, lumbar vein, cava, pubop...
           muscle_capsule              [muscle, capsule, detrusor fibers, capsule fib...
           prostate_tissue             [prostate, adenoma, prostate contour, tumor, m...
           seminal_vesicle_vas         [Seminal Vesicles, vas deferens, tissue, Vas d...
           ureter                         [ureter, gondal, gonadal, colon, fat, urether]
           urethra                     [urethra, bladder, sphincter, lateral structur...

## Evaluate Clustering

In [82]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary
import re
from collections import defaultdict

def get_df_tokens(doc2cluster: dict[str, str]):
    rows = []
    for doc, cluster in doc2cluster.items():
        rows.append({
            "doc": doc,
            "cluster": cluster
        })
    df = pd.DataFrame(rows)
    df['tokens'] = df['doc'].apply(lambda x: re.findall(r"[a-z]+", x.lower().replace('[', '').replace(']', '').split(',')[0]))
    return df

def get_topics(doc2cluster: dict[str, str], n_top_words: int = 10):
    df = get_df_tokens(doc2cluster)
    dictionary = Dictionary(df['tokens'])
    corpus = [dictionary.doc2bow(doc) for doc in df['tokens']]
    cluster_to_tokens = defaultdict(list)
    for toks, c in zip(df['tokens'], df['cluster']):
        cluster_to_tokens[c].extend(toks)

    cluster_topics = []
    cluster_labels = []  # optional: to keep original ids
    for c, toks in cluster_to_tokens.items():
        counts = Counter(toks)
        top_words = [w for w, _ in counts.most_common(n_top_words)]
        cluster_topics.append(top_words)
        cluster_labels.append(c)
    return cluster_topics, cluster_labels, dictionary, corpus

def compute_coherence_gensim(doc2cluster: dict[str, str], n_top_words: int = 10):
    df = get_df_tokens(doc2cluster)
    cluster_topics, cluster_labels, dictionary, corpus = get_topics(doc2cluster, n_top_words)

    # 4) Compute coherence
    # Recommended: 'c_v' or 'c_npmi' (needs texts). 'u_mass' needs corpus+dictionary.
    cm_cv = CoherenceModel(
        topics=cluster_topics,
        texts=df["tokens"],          # required for c_v / c_npmi
        dictionary=dictionary,
        coherence='c_v',
        corpus=corpus
    )   
    return cm_cv

from octis.evaluation_metrics.coherence_metrics import WECoherencePairwise, WECoherenceCentroid

def compute_coherence_emb_pairwise(doc2cluster: dict[str, str], n_top_words: int = 10):
    cluster_topics, cluster_labels, dictionary, corpus = get_topics(doc2cluster, n_top_words)
    for topic in cluster_topics:
        print(len(topic))
    metric = WECoherencePairwise(topk=n_top_words)
    coherence = metric.score({'topics': cluster_topics})
    return coherence

def compute_coherence_emb_centroid(doc2cluster: dict[str, str], n_top_words: int = 10):
    cluster_topics, cluster_labels, dictionary, corpus = get_topics(doc2cluster, n_top_words)
    for topic in cluster_topics:
        print(len(topic))
    metric = WECoherenceCentroid(topk=n_top_words)
    coherence = metric.score({'topics': cluster_topics})
    return coherence


### 1) BERTopic Clustering

In [83]:
i_doc2cluster_path = '/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/urology-related/rafal_instrument_clusters.csv'
a_doc2cluster_path = i_doc2cluster_path.replace('instrument', 'action')
t_doc2cluster_path = i_doc2cluster_path.replace('instrument', 'tissue')

i_doc2cluster = pd.read_csv(i_doc2cluster_path)[['Document', 'Name']]
i_doc2cluster = dict(zip(i_doc2cluster['Document'], i_doc2cluster['Name']))

a_doc2cluster = pd.read_csv(a_doc2cluster_path)[['Document', 'Name']]
a_doc2cluster = dict(zip(a_doc2cluster['Document'], a_doc2cluster['Name']))

t_doc2cluster = pd.read_csv(t_doc2cluster_path)[['Document', 'Name']]
t_doc2cluster = dict(zip(t_doc2cluster['Document'], t_doc2cluster['Name']))

print("INSTRUMENT:")
i_cm_cv = compute_coherence_gensim(i_doc2cluster)
print(i_cm_cv.get_coherence())
print('-'*20)
print("ACTION:")
a_cm_cv = compute_coherence_gensim(a_doc2cluster)
print(a_cm_cv.get_coherence())
print('-'*20)
print("TISSUE:")
t_cm_cv = compute_coherence_gensim(t_doc2cluster)
print(t_cm_cv.get_coherence())

bertopic_clustering_metrics = {'method': 'bertopic_clustering'}
bertopic_clustering_metrics['instrument'] = round(i_cm_cv.get_coherence(), 2)
bertopic_clustering_metrics['action'] = round(a_cm_cv.get_coherence(), 2)
bertopic_clustering_metrics['tissue'] = round(t_cm_cv.get_coherence(), 2)

INSTRUMENT:


0.5167408451332131
--------------------
ACTION:
0.44590681014655376
--------------------
TISSUE:
0.4221840024296348


### 2) Initial LLM Clustering

In [87]:
import json

i_doc2cluster_path = '/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/urology-related/o3_instrument_clusters-mapping.json'
a_doc2cluster_path = i_doc2cluster_path.replace('instrument', 'action')
t_doc2cluster_path = i_doc2cluster_path.replace('instrument', 'tissue')

with open(i_doc2cluster_path, 'r') as f:
    i_doc2cluster = json.load(f)

with open(a_doc2cluster_path, 'r') as f:
    a_doc2cluster = json.load(f)

with open(t_doc2cluster_path, 'r') as f:
    t_doc2cluster = json.load(f)

print("INSTRUMENT:")
i_cm_cv = compute_coherence_gensim(i_doc2cluster)
print(i_cm_cv.get_coherence())
print('-'*20)
print("ACTION:")
a_cm_cv = compute_coherence_gensim(a_doc2cluster)
print(a_cm_cv.get_coherence())
print('-'*20)
print("TISSUE:")
t_cm_cv = compute_coherence_gensim(t_doc2cluster)
print(t_cm_cv.get_coherence())

initial_llm_clustering_metrics = {'method': 'initial_llm_clustering'}
initial_llm_clustering_metrics['instrument'] = round(i_cm_cv.get_coherence(), 2)
initial_llm_clustering_metrics['action'] = round(a_cm_cv.get_coherence(), 2)
initial_llm_clustering_metrics['tissue'] = round(t_cm_cv.get_coherence(), 2)

INSTRUMENT:
0.553387295836158
--------------------
ACTION:
0.5039468094095351
--------------------
TISSUE:
0.4725659349653062


### 3) Final LLM Clustering

In [88]:
annotations_df = pd.read_csv('/home/firdavs/surgery/surgical_fb_generation/SurgFBGen/data/iat_predictor_splits/full.csv')

i_doc2cluster = dict(zip(annotations_df['instrument-extraction'], annotations_df['instrument']))
a_doc2cluster = dict(zip(annotations_df['action-extraction'], annotations_df['action']))
t_doc2cluster = dict(zip(annotations_df['tissue-extraction'], annotations_df['tissue']))

print("INSTRUMENT:")
i_cm_cv = compute_coherence_gensim(i_doc2cluster)
print(i_cm_cv.get_coherence())
print('-'*20)
print("ACTION:")
a_cm_cv = compute_coherence_gensim(a_doc2cluster)
print(a_cm_cv.get_coherence())
print('-'*20)
print("TISSUE:")
t_cm_cv = compute_coherence_gensim(t_doc2cluster)
print(t_cm_cv.get_coherence())

final_llm_clustering_metrics = {'method': 'final_llm_clustering'}
final_llm_clustering_metrics['instrument'] = round(i_cm_cv.get_coherence(), 2)
final_llm_clustering_metrics['action'] = round(a_cm_cv.get_coherence(), 2)
final_llm_clustering_metrics['tissue'] = round(t_cm_cv.get_coherence(), 2)

INSTRUMENT:


0.5116331798894008
--------------------
ACTION:
0.49025317055470763
--------------------
TISSUE:
0.5684631273013405


### 4) All

In [69]:
clustering_coherence_df = pd.DataFrame([bertopic_clustering_metrics, initial_llm_clustering_metrics, final_llm_clustering_metrics]).round(2)
clustering_coherence_df['instrument-gain'] = clustering_coherence_df.apply(
    lambda row: row['instrument'] / bertopic_clustering_metrics['instrument'] - 1,
    axis=1
)
clustering_coherence_df['action-gain'] = clustering_coherence_df.apply(
    lambda row: row['action'] / bertopic_clustering_metrics['action'] - 1,
    axis=1
)
clustering_coherence_df['tissue-gain'] = clustering_coherence_df.apply(
    lambda row: row['tissue'] / bertopic_clustering_metrics['tissue'] - 1,
    axis=1
)
clustering_coherence_df = clustering_coherence_df[['method', 'instrument', 'instrument-gain', 'action', 'action-gain', 'tissue', 'tissue-gain']].round(2)
clustering_coherence_df

,method,instrument,instrument-gain,action,action-gain,tissue,tissue-gain
0,bertopic_clustering,0.49,0.00,0.56,0.00,0.50,0.00
1,initial_llm_clustering,0.58,0.18,0.59,0.05,0.55,0.10
2,final_llm_clustering,0.55,0.12,0.66,0.18,0.68,0.36


In [89]:
clustering_coherence_df = pd.DataFrame([bertopic_clustering_metrics, initial_llm_clustering_metrics, final_llm_clustering_metrics]).round(2)
clustering_coherence_df['instrument-gain'] = clustering_coherence_df.apply(
    lambda row: row['instrument'] / bertopic_clustering_metrics['instrument'] - 1,
    axis=1
)
clustering_coherence_df['action-gain'] = clustering_coherence_df.apply(
    lambda row: row['action'] / bertopic_clustering_metrics['action'] - 1,
    axis=1
)
clustering_coherence_df['tissue-gain'] = clustering_coherence_df.apply(
    lambda row: row['tissue'] / bertopic_clustering_metrics['tissue'] - 1,
    axis=1
)
clustering_coherence_df = clustering_coherence_df[['method', 'instrument', 'instrument-gain', 'action', 'action-gain', 'tissue', 'tissue-gain']].round(2)
clustering_coherence_df

,method,instrument,instrument-gain,action,action-gain,tissue,tissue-gain
0,bertopic_clustering,0.52,0.00,0.45,0.00,0.42,0.00
1,initial_llm_clustering,0.55,0.06,0.50,0.11,0.47,0.12
2,final_llm_clustering,0.51,-0.02,0.49,0.09,0.57,0.36
